# Reproduction

This notebook currently imports results from a run of PHC.py and attempts to map them to Table 6.

Table 6 results are from Shoaib M, Ramamohan V. **Simulation modeling and analysis of primary health center operations**. *SIMULATION* 98(3):183-208. (2022). <https://doi.org/10.1177/00375497211030931>.

## Set up

In [1]:
# To run model
import PHC
import numpy as np

# To import and process results
import pandas as pd
import xlrd
import os

In [2]:
table6_path = '../original_study/tab6.csv'

## Run model

Set parameters for each configuration based on Table 3.

In [3]:
# For all: days 365, warm up 180, replication 10, 6 inpatient beds, 1 delivery
# bed, 3 staff nurse, 1 NCD nurse. These are default parameters in the model,
# and below, have just specified the parameters that changed between
# configurations

# TODO: Change all replication to 100 (just running with 10 for now for speed)

t6_c1_param = {
    'doc_cap': 2,
    'OPD_iat': 4,
    'IPD_iat': 2880,
    'delivery_iat': 1440,
    'ANC_iat': 1440,
    'rep_file': 't6_c1.xls',
    'replication': 2  # TODO: Change to 10
}

t6_c2_param = {
    'doc_cap': 1,
    'OPD_iat': 9,
    'IPD_iat': 2880,
    'delivery_iat': 2880,
    'ANC_iat': 2880,
    'rep_file': 't6_c2.xls',
    'replication': 2  # TODO: Change to 10
}

t6_c3_param = {
    'doc_cap': 1,
    'OPD_iat': 9,
    'IPD_iat': 2880,
    'delivery_iat': np.inf,
    'ANC_iat': np.inf,
    'rep_file': 't6_c3.xls'
}

t6_c4_param = {
    'doc_cap': 2,
    'OPD_iat': 3,
    'IPD_iat': 2880,
    'delivery_iat': 1440,
    'ANC_iat': 1440,
    'rep_file': 't6_c4.xls'
}

Check config 1 has same global values after being run twice

In [4]:
# Input configuration parameters to main() (have to append with 's_', and add
# ** as inputting dict as parameters). Then run the model

# Run the function twice, saving the global variables each time

# Run 1 of config 1
PHC.main(**{f's_{k}': v for k, v in t6_c1_param.items()})
globals_a = PHC.return_globals()

# Run 2 of config 1
PHC.main(**{f's_{k}': v for k, v in t6_c1_param.items()})
globals_b = PHC.return_globals()

# Confirm if parameters are consistent
globals_a == globals_b

 No of replications done 0
 No of replications done 1
 No of replications done 0
 No of replications done 1


True

Check config 2 has the same global values being run before and after having run config 1

In [5]:
# Run 1 of config 2
PHC.main(**{f's_{k}': v for k, v in t6_c2_param.items()})
globals_a = PHC.return_globals()

# Intermediate run of config 1
PHC.main(**{f's_{k}': v for k, v in t6_c1_param.items()})

# Run 2 of config 2
PHC.main(**{f's_{k}': v for k, v in t6_c2_param.items()})
globals_b = PHC.return_globals()

# Confirm if parameters are consistent
globals_a == globals_b

 No of replications done 0
 No of replications done 1
 No of replications done 0
 No of replications done 1
 No of replications done 0
 No of replications done 1


True

In [6]:
# View example of output from this
globals_a

{'__name__': 'PHC',
 '__doc__': None,
 '__package__': '',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x7067cc291bb0>,
 '__spec__': ModuleSpec(name='PHC', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7067cc291bb0>, origin='/home/amy/Documents/stars/stars-reproduce-shoaib-2022/reproduction/PHC.py'),
 '__file__': '/home/amy/Documents/stars/stars-reproduce-shoaib-2022/reproduction/PHC.py',
 '__cached__': '/home/amy/Documents/stars/stars-reproduce-shoaib-2022/reproduction/__pycache__/PHC.cpython-39.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'),
  '__build_class__': <function __build_class__>,
  '__import__': <function __import

In [7]:
globals_b

{'__name__': 'PHC',
 '__doc__': None,
 '__package__': '',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x7067cc291bb0>,
 '__spec__': ModuleSpec(name='PHC', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7067cc291bb0>, origin='/home/amy/Documents/stars/stars-reproduce-shoaib-2022/reproduction/PHC.py'),
 '__file__': '/home/amy/Documents/stars/stars-reproduce-shoaib-2022/reproduction/PHC.py',
 '__cached__': '/home/amy/Documents/stars/stars-reproduce-shoaib-2022/reproduction/__pycache__/PHC.cpython-39.pyc',
 '__builtins__': {'__name__': 'builtins',
  '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.",
  '__package__': '',
  '__loader__': _frozen_importlib.BuiltinImporter,
  '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in'),
  '__build_class__': <function __build_class__>,
  '__import__': <function __import

## Import and process replication results

# Make dictionary with labels from table 6, and corresponding names from model output
t6_labels = {
  'doc occ': 'Doctor utilisation',
  'NCD occ': 'NCD Nurse utilisation',
  'staff nurse occ': 'Staff nurse utilisation',
  'pharm occ': 'Pharmacist utilisation',
  'lab occ': 'Lab utilisation',
  'ipd bed occ': 'Inpatient bed utilisation',
  'del occ': 'Labour bed utilisation',  # "Del" stands for delivery
  'OPD q len': 'Mean length of OPD queue (number of patients)',
  'OPD Q wt': 'OPD queue waiting time (minutes)',
  'pharmacy q len': 'Mean length of pharmacy queue (number of patients)',
  'Pharmacy Q wt': 'Pharmacy queue waiting time (minutes)',
  'lab q len': 'Mean length of Lab queue (number of patients)',
  'Lab Q wt': 'Lab queue waiting time (minutes)',
  'prop_del_referred': 'Fraction of childbirth cases referred'
}

# List of files to loop through
files = ['t6_c1', 't6_c2']
#files = ['t6_c1', 't6_c2', 't6_c3', 't6_c4']

# Empty list to store results
result_list = []

for f in files:
    # Import .xls and convert to pandas dataframe
    book = xlrd.open_workbook(os.path.join('outputs', f'{f}.xls'))
    result = pd.read_excel(book, header=None, index_col=0)

    # Find mean from the replications
    av_result = result.mean(axis=1)

    # Calculate the proportion of childbirth cases referred and add to series
    prop_del_referred = pd.Series(
        av_result['del referred'] / av_result['Del patients'],
        index=['prop_del_referred'])
    new_av_result = pd.concat([av_result, prop_del_referred])

    # Convert to df, save as model config1, drop duplicates, and display
    res = (new_av_result
           .to_frame(name=f'model_{f}')
           #.reset_index()
           #.rename(columns= {'index': 'simple_outcome'})
           .drop_duplicates())

    result_list.append(res)

# Combine into single dataframe
summary = (pd.concat(result_list, axis=1)
           .reset_index()
           .rename(columns= {'index': 'model_outcome'}))

# Add labels to model results
summary['t6_outcome'] = summary['model_outcome'].map(t6_labels)

summary

## Import table 6 results and compare against run results

# Import table 6
t6 = pd.read_csv(table6_path)
t6_c1 = t6[['outcome', 'config1_mean', 'config2_mean']].rename(columns={
    'outcome': 't6_outcome',
    'config1_mean': 'model_t6_c1',
    'config2_mean': 'model_t6_c2'
})
t6

# Merge and set column order
compare = t6_c1.merge(simple_result)[['t6_outcome', 'simple_outcome',
                                      't6_result', 'simple_result',]]

compare

compare.to_csv('outputs/t6_compare.csv', index=False)